In [1]:
import os
# These are the default AWS configurations for the Analysis Sandbox.
# that are set in the environmnet variables. 
aws_default_config = {
    #'AWS_NO_SIGN_REQUEST': 'YES', 
    'AWS_SECRET_ACCESS_KEY': 'fake',
    'AWS_ACCESS_KEY_ID': 'fake',
}

# To access public bucket, need to remove the AWS credentials in 
# the environment variables or the following error will occur.
# PermissionError: The AWS Access Key Id you provided does not exist in our records.

for key in aws_default_config.keys():
    if key in os.environ:
        del os.environ[key]

In [2]:
import logging
import os

import click
import fsspec
import geopandas as gpd
import numpy as np
import rasterio.features
from datacube.utils.cog import to_cog
from datacube.utils.dask import save_blob_to_file, save_blob_to_s3
from datacube.utils.geometry import Geometry
from odc.dscache.tools.tiling import parse_gridspec_with_name
from odc.geo.geobox import GeoBox
from odc.geo.xr import wrap_xr
from pandas.api.types import is_float_dtype, is_integer_dtype

from deafrica_conflux.cli.logs import logging_setup
from deafrica_conflux.filter_polygons import get_intersecting_polygons
from deafrica_conflux.id_field import guess_id_field
from deafrica_conflux.io import check_dir_exists, check_file_exists, check_if_s3_uri

In [3]:
verbose = 1
# Grid name africa_{10|20|30|60}
grid_name = "africa_30"
# Datacube product to get tiles for from the grid.
product = "wofs_ls"
# Path to the polygons to be rasterised.
polygons_file_path = "s3://deafrica-waterbodies-dev/waterbodies/v0.0.2/senegal_basin/historical_extent/waterbodies.parquet"
# Unique key id polygons vector file
use_id = "WB_ID"
# Directory to write the tiled polygon rasters to.
output_directory = "s3://deafrica-waterbodies-dev/waterbodies/v0.0.2/senegal_basin/conflux/"
# Overwrite existing polygons raster file
overwrite = True

In [4]:
# Set up logger.
logging_setup(verbose)
_log = logging.getLogger(__name__)

In [5]:
# Support pathlib Paths.
polygons_file_path = str(polygons_file_path)
output_directory = str(output_directory)

In [6]:
# Create the output directory if it does not exist.
is_s3 = check_if_s3_uri(output_directory)
if is_s3:
    fs = fsspec.filesystem("s3")
else:
    fs = fsspec.filesystem("file")

if not check_dir_exists(output_directory):
    fs.makedirs(output_directory, exist_ok=True)
    _log.info(f"Created directory {output_directory}")

tiles_output_directory = os.path.join(output_directory, "product_tiles")
if not check_dir_exists(tiles_output_directory):
    fs.makedirs(tiles_output_directory, exist_ok=True)
    _log.info(f"Created directory {tiles_output_directory}")

rasters_output_directory = os.path.join(output_directory, "historical_extent_rasters")
if not check_dir_exists(rasters_output_directory):
    fs.mkdirs(rasters_output_directory, exist_ok=True)
    _log.info(f"Created directory {output_directory}")

[2023-12-04 09:56:11,268] {credentials.py:611} INFO - Found credentials in shared credentials file: ~/.aws/credentials


In [7]:
# Get the GridSpec.
grid, gridspec = parse_gridspec_with_name(grid_name)
_log.info(f"Using the grid {grid} with {gridspec}")

# From the GridSpec get the crs and resolution.
crs = gridspec.crs
resolution = abs(gridspec.resolution[0])

# Read the product footprint.
product_footprint = gpd.read_file(
    f"https://explorer.digitalearth.africa/api/footprint/{product}"
).to_crs(crs)
# Get the product footprint geopolygon.
product_footprint = Geometry(geom=product_footprint.geometry[0], crs=crs)

# Get the tiles covering the product footprint.
tiles = gridspec.tiles_from_geopolygon(geopolygon=product_footprint)
tiles = list(tiles)

# Get the individual tile geometries.
tile_geometries = []
tile_ids = []
for tile in tiles:
    tile_idx, tile_idy = tile[0]
    tile_geometry = tile[1].extent.geom

    tile_geometries.append(tile_geometry)
    tile_ids.append(f"x{tile_idx:03d}_y{tile_idy:03d}")

tiles_gdf = gpd.GeoDataFrame(data={"tile_ids": tile_ids, "geometry": tile_geometries}, crs=crs)
_log.info(f"Tile count: {len(tiles_gdf)}")

tiles_output_fp = os.path.join(tiles_output_directory, f"{product}_tiles.parquet")
tiles_gdf.to_parquet(tiles_output_fp)
_log.info(f"{product} tiles written to {tiles_output_fp}")

[2023-12-04 09:56:11,482] {2832167676.py:3} INFO - Using the grid africa_30 with GridSpec(crs=epsg:6933, tile_size=(96000.0, 96000.0), resolution=(-30, 30))
[2023-12-04 09:56:15,158] {2832167676.py:31} INFO - Tile count: 6010
[2023-12-04 09:56:15,368] {2832167676.py:35} INFO - wofs_ls tiles written to s3://deafrica-waterbodies-dev/waterbodies/v0.0.2/senegal_basin/conflux/product_tiles/wofs_ls_tiles.parquet


In [8]:
# Load the polygons.
polygons_gdf = gpd.read_parquet(polygons_file_path).to_crs(crs)
_log.info(f"Polygon count {len(polygons_gdf)}")

[2023-12-04 09:56:15,806] {2026362536.py:3} INFO - Polygon count 8768


In [9]:
# Check the id column is unique.
id_field = guess_id_field(input_gdf=polygons_gdf, use_id=use_id)

[2023-12-04 09:56:15,814] {id_field.py:64} INFO - Values in the column WB_ID are unique.


In [10]:
assert is_integer_dtype(polygons_gdf[id_field]) or is_float_dtype(polygons_gdf[id_field])

In [11]:
_log.info("Filtering out tiles that do not intersect with any polygon...")
filtered_tiles_gdf = get_intersecting_polygons(
    region=polygons_gdf, polygons_gdf=tiles_gdf, use_id="tile_ids"
)
_log.info(f"Filtered out {len(tiles_gdf) - len(filtered_tiles_gdf)} tiles.")
_log.info(f"Filtered tiles count: {len(filtered_tiles_gdf)}.")

[2023-12-04 09:56:15,827] {4261533379.py:1} INFO - Filtering out tiles that do not intersect with any polygon...
[2023-12-04 09:56:15,830] {id_field.py:64} INFO - Values in the column tile_ids are unique.
[2023-12-04 09:56:15,889] {4261533379.py:5} INFO - Filtered out 5950 tiles.
[2023-12-04 09:56:15,889] {4261533379.py:6} INFO - Filtered tiles count: 60.


In [12]:
# Split each row in the tiles into a GeoDataFrame of its own.
tiles = np.array_split(filtered_tiles_gdf, len(filtered_tiles_gdf))
assert len(tiles) == len(filtered_tiles_gdf)

for i, tile in enumerate(tiles):
    tile_id = tile["tile_ids"].iloc[0]
    tile_geometry = tile.geometry.iloc[0]
    _log.info(f"Rasterizing polygons for tile {tile_id} ({i + 1}/{len(tiles)})")

    tile_raster_fp = os.path.join(rasters_output_directory, f"{tile_id}.tif")

    if not overwrite:
        _log.info(f"Checking existence of {tile_raster_fp}")
        exists = check_file_exists(tile_raster_fp)

    if overwrite or not exists:
        # Get the geobox for the region.
        tile_geobox = GeoBox.from_geopolygon(
            geopolygon=Geometry(geom=tile_geometry, crs=crs),
            resolution=resolution,
            crs=crs,
        )

        # Get the polygons that intersect with the tile.
        tile_polygons = get_intersecting_polygons(
            region=tile, polygons_gdf=polygons_gdf, use_id=id_field
        )

        # Rasterise shapes into a numpy array
        shapes = zip(tile_polygons.geometry, tile_polygons[id_field])
        tile_raster_np = rasterio.features.rasterize(
            shapes=shapes, out_shape=tile_geobox.shape, transform=tile_geobox.transform
        )

        # Convert numpy array to a full xarray.DataArray
        tile_raster_ds = wrap_xr(im=tile_raster_np, gbox=tile_geobox)

        # Write the raster to disk.
        cog_bytes = to_cog(tile_raster_ds)
        if check_if_s3_uri(tile_raster_fp):
            save_blob_to_s3(data=cog_bytes, url=tile_raster_fp).compute()
        else:
            save_blob_to_file(data=cog_bytes, url=tile_raster_fp).compute()
        _log.info(f"Exported raster data to {tile_raster_fp}")
    else:
        _log.info(f"{tile_raster_fp} already exists, skipping...")

[2023-12-04 09:56:16,118] {3176611446.py:8} INFO - Rasterizing polygons for tile x168_y090 (1/60)
[2023-12-04 09:56:16,121] {id_field.py:64} INFO - Values in the column WB_ID are unique.
[2023-12-04 09:56:16,768] {3176611446.py:44} INFO - Exported raster data to s3://deafrica-waterbodies-dev/waterbodies/v0.0.2/senegal_basin/conflux/historical_extent_rasters/x168_y090.tif
[2023-12-04 09:56:16,769] {3176611446.py:8} INFO - Rasterizing polygons for tile x169_y090 (2/60)
[2023-12-04 09:56:16,771] {id_field.py:64} INFO - Values in the column WB_ID are unique.
[2023-12-04 09:56:17,087] {3176611446.py:44} INFO - Exported raster data to s3://deafrica-waterbodies-dev/waterbodies/v0.0.2/senegal_basin/conflux/historical_extent_rasters/x169_y090.tif
[2023-12-04 09:56:17,088] {3176611446.py:8} INFO - Rasterizing polygons for tile x168_y091 (3/60)
[2023-12-04 09:56:17,089] {id_field.py:64} INFO - Values in the column WB_ID are unique.
[2023-12-04 09:56:17,406] {3176611446.py:44} INFO - Exported rast